# Split atmospheric profiles in the differents filters

- author  :Sylvie Dagoret-Campagne
- affiliaiton : IJCLab/IN2P3/CNRS
- creation date : April 24th 2020
- update date :


In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import numpy as np
import pandas as pd
import os
import sys
import re
%matplotlib inline

In [2]:
from astropy.io import fits

In [3]:
#to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10,6),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [4]:
plt.rcParams.update({'figure.max_open_warning': 0}) # to see a large number of files

In [5]:
%load_ext autoreload
%autoreload 2

# Configuration

## Wavelength bins

In [6]:
file_wavelengthbins="data/WaveLengthBins.csv"

In [7]:
df_wl=pd.read_csv(file_wavelengthbins,index_col=0)

In [8]:
df_wl

,band,wl_aver,wl_wdth,wl_min,wl_max
0,u,356.0,78.0,317.0,395.0
1,g,472.0,154.0,395.0,549.0
2,r,619.0,140.0,549.0,689.0
3,i,754.0,130.0,689.0,819.0
4,z,870.5,103.0,819.0,922.0
5,y,1010.0,176.0,922.0,1098.0


In [9]:
NBFILTER=len(df_wl)

In [10]:
FILTER_NAME=df_wl["band"].values
WL_MIN=df_wl["wl_min"].values
WL_MAX=df_wl["wl_max"].values

## Absorption patterns

In [11]:
file_abspattern="data/absorption_pattern.csv"

In [12]:
df_atm=pd.read_csv(file_abspattern,index_col=0)

In [13]:
df_atm

,wl,o2_10,o2_11,o2_12,o2_13,o2_14,o2_15,o2_16,o2_17,o2_18,...,no2_16,no2_17,no2_18,no2_19,no2_20,no2_21,no2_22,no2_23,no2_24,no2_25
0,250.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.999785,0.999772,0.999758,0.999745,0.999731,0.999718,0.999704,0.999691,0.999678,0.999664
1,251.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.999841,0.999831,0.999821,0.999811,0.999801,0.999791,0.999781,0.999771,0.999761,0.999752
2,252.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.999849,0.999839,0.999830,0.999821,0.999811,0.999802,0.999792,0.999783,0.999774,0.999764
3,253.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.999842,0.999832,0.999822,0.999812,0.999802,0.999792,0.999782,0.999773,0.999763,0.999753
4,254.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.999833,0.999823,0.999813,0.999802,0.999792,0.999782,0.999771,0.999761,0.999750,0.999740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,1196.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
947,1197.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
948,1198.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
949,1199.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Split the rows in different filters

In [14]:
wl=df_atm["wl"]

In [15]:
all_wl_indexes=[]
for iband in np.arange(NBFILTER):
    wl_indexes=np.where(np.logical_and(wl>=WL_MIN[iband],wl<=WL_MAX[iband]))[0]
    all_wl_indexes.append(wl_indexes)

In [16]:
all_df=[]
for iband in np.arange(NBFILTER):
    df=df_atm.iloc[all_wl_indexes[iband]]
    all_df.append(df)


In [17]:
all_df[5]

,wl,o2_10,o2_11,o2_12,o2_13,o2_14,o2_15,o2_16,o2_17,o2_18,...,no2_16,no2_17,no2_18,no2_19,no2_20,no2_21,no2_22,no2_23,no2_24,no2_25
672,922.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
673,923.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
674,924.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
675,925.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
676,926.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,1094.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
845,1095.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
846,1096.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
847,1097.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# Save file

In [18]:
rootfilename=file_abspattern.split(".")[0]

In [19]:
rootfilename

'data/absorption_pattern'

In [20]:
for iband in np.arange(NBFILTER):
    filename=rootfilename+"_"+FILTER_NAME[iband]+".csv"
    all_df[iband].to_csv(filename)    

# 5 Check

In [21]:
for iband in np.arange(NBFILTER):
    filename=rootfilename+"_"+FILTER_NAME[iband]+".csv"
    print("====================",filename,"=======================")
    df=pd.read_csv(filename,index_col=0)
    df=df.reset_index(drop=True)
    print(df)

==================== data/absorption_pattern_u.csv =======================
       wl  o2_10  o2_11  o2_12  o2_13  o2_14  o2_15  o2_16  o2_17  o2_18  ...  \
0   317.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
1   318.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
2   319.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
3   320.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
4   321.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
..    ...    ...    ...    ...    ...    ...    ...    ...    ...    ...  ...   
74  391.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
75  392.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
76  393.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
77  394.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  ...   
78  395.0    1.0    1.0    1.0    